In [52]:
import importlib
import os
from models import *
from dataprocessor import *
def reload_custom_libs():
    import loss_functions
    import models
    import dataprocessor
    importlib.reload(loss_functions)
    importlib.reload(models)
    importlib.reload(dataprocessor)

In [77]:
def check_for_nan(tensors):
    """检查给定张量列表中是否有NaN值"""
    for tensor in tensors:
        if tf.reduce_any(tf.math.is_nan(tensor)):
            print("NAN in dataset")
            return True
        if tf.reduce_any(tf.math.is_inf(tensor)):
            print("inf in dataset")  
            return True
    return False

In [110]:
macd_timescales = [(8, 24), (16, 28), (32, 96)]
rtn_timescales = [1, 21, 63, 126, 252]
timesteps = 126

folder_path = 'data'
files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]
files = [file for file in files if file not in ("CC00.NYB.xlsx", "LB00.CME.xlsx", "ES00.CME.xlsx", "NQ00.CME.xlsx", "YM00.CBT.xlsx", "SP00.CME.xlsx")]
data_list = []


data_list = process_data_list(files, macd_timescales, rtn_timescales, test = False)
target_set, labels, map = generate_tensors(data_list, timesteps, encoder_type = "one-hot", return_map=True)
context_set, _ = generate_tensors(data_list, timesteps, encoder_type = "one-hot", contain_next_day_rtn=True)

生成张量, 并对类别信息进行on-hot 编码: 100%|██████████| 11/11 [00:34<00:00,  3.10s/it]


one-hot 编码中...


生成张量, 并对类别信息进行on-hot 编码: 100%|██████████| 11/11 [00:33<00:00,  3.02s/it]


one-hot 编码中...


In [111]:
batch_size = len(files)
data_set = data_binder(context_set, target_set, labels, batch_size=batch_size)
data_set.save(f"saved_data")

dataset = tf.data.Dataset.load("saved_data")

处理日期中..:  74%|███████▍  | 64271/86898 [27:18<09:23, 40.18it/s]

In [ ]:
reload_custom_libs()
from models import *
from dataprocessor import *
from loss_functions import *
# from heartrate import trace

hidden_dim = 64 # 128
warm_up = 63
target_std = tf.cast(5e-2, tf.float64)
features_len = len(macd_timescales) + len(rtn_timescales)
encoding_size = len(files) + 1
x_shape = (batch_size, timesteps, features_len)
s_shape = (batch_size, timesteps, encoding_size)

model = ModelWrapper(features_len, hidden_dim, encoding_size, num_heads = 4)
model.build((x_shape, s_shape))

In [ ]:
if tf.reduce_any(tf.math.is_inf(x)):
    print("NAN in dataset")

NAN in dataset


In [ ]:
x

(<tf.Tensor: shape=(12, 126, 9), dtype=float32, numpy=
 array([[[-1.87198073e-02,  1.46927214e+00,  1.56255877e+00, ...,
           6.28592551e-01,  3.25806856e-01, -8.83754790e-01],
         [-2.46153842e-03,  1.37680817e+00,  1.51414728e+00, ...,
           6.96697891e-01,  3.59317958e-01, -9.36222613e-01],
         [-2.77606416e-02,  1.23691380e+00,  1.43545115e+00, ...,
           6.95032299e-01,  3.30505311e-01, -1.01555002e+00],
         ...,
         [-2.03252025e-02,  7.67160594e-01,  4.78691816e-01, ...,
           2.64380842e-01, -6.06546998e-01, -2.18781292e-01],
         [-1.38312590e-03,  7.74119854e-01,  5.12650788e-01, ...,
           8.77545401e-02, -7.64421642e-01, -2.43626878e-01],
         [-6.23268681e-03,  5.90721011e-01,  4.23517406e-01, ...,
           1.09790146e-01, -8.53416383e-01, -2.51632184e-01]],
 
        [[-1.21951215e-02,  7.55662143e-01,  9.75598991e-01, ...,
           1.76995945e+00,  5.10290563e-01, -6.45975351e-01],
         [ 8.91632400e-03,  6.93

In [ ]:
iter_count, num = 0, 0
for batch_data in tqdm(dataset, desc=f"训练中...epoch"):
    x_c, x_c_rtn, s_c, x, s, rtn_std, _, _ = batch_data
    if check_for_nan([x_c, x_c_rtn, s_c, s, rtn_std]):
        num += 1
        print(f"detected in batch_data, iteration {iter_count}")
    iter_count += 1

训练中...epoch: 100%|██████████| 2362/2362 [00:02<00:00, 900.95it/s]


In [ ]:
num_epochs = 100
print_interval = 10

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

def train():
    
    dataset = tf.data.Dataset.load("saved_data")
    for epoch in range(num_epochs):
        dataset = dataset.shuffle(buffer_size=10000)
        iter_count = 0
        for batch_data in tqdm(dataset, desc=f"训练中...epoch{epoch}"):
            x_c, x_c_rtn, s_c, x, s, rtn_std, _, _ = batch_data
            if check_for_nan([x_c, x_c_rtn, s_c, x, s, rtn_std]):
                print(f"NaN or inf detected in batch_data, iteration {iter_count}")
                return batch_data
            with tf.GradientTape() as tape:
                result = model(x_c, x_c_rtn, s_c, x, s)
                if check_for_nan(result):
                    print(f"NaN detected in result, iteration {iter_count}")
                    return batch_data, result
                loss, mle, sharpe = joint_loss_function(result, rtn_std, target_std, warm_up, alpha=1)
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
            iter_count += 1
            if iter_count % 1 == 0:
                print(f"Epoch {epoch+1}/{num_epochs}, Iteration {iter_count}, Loss: {loss.numpy()}, mle: {mle}, sharpe: {sharpe}")

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.numpy()}")
    
result = train()

训练中...epoch0:   0%|          | 1/2362 [00:03<2:02:39,  3.12s/it]

Epoch 1/100, Iteration 1, Loss: 1422.1148424899147, mle: 1424.1230758264785, sharpe: -2.008233336563651


训练中...epoch0:   0%|          | 2/2362 [00:05<1:34:04,  2.39s/it]

Epoch 1/100, Iteration 2, Loss: 1033.3363902679719, mle: 1033.6609271220352, sharpe: -0.32453685406331934


训练中...epoch0:   0%|          | 3/2362 [00:07<1:27:42,  2.23s/it]

...

Epoch 1/100, Iteration 6, Loss: 139.421438704546, mle: 136.45181725785892, sharpe: 2.9696214466870883


训练中...epoch0:   0%|          | 7/2362 [00:14<1:17:01,  1.96s/it]

Epoch 1/100, Iteration 7, Loss: 1265.911528778797, mle: 1265.4257165296929, sharpe: 0.48581224910409854


训练中...epoch0:   0%|          | 7/2362 [00:16<1:31:11,  2.32s/it]


KeyboardInterrupt: 

In [ ]:
result

(<tf.Tensor: shape=(12, 126, 2), dtype=float64, numpy=
 array([[[6.11744404, 1.29992247],
         [6.05334282, 1.19234073],
         [6.04828453, 1.12541842],
         ...,
         [6.06719971, 0.9283601 ],
         [6.0078578 , 0.92848593],
         [6.02805185, 0.93377763]],
 
        [[1.25681174, 1.21873188],
         [1.18810105, 1.17738163],
         [1.14196587, 1.16670406],
         ...,
         [1.28490782, 1.0845989 ],
         [1.2713486 , 1.11892009],
         [1.2127372 , 1.10908401]],
 
        [[4.17370129, 1.50630212],
         [4.08983517, 1.42670906],
         [4.06532669, 1.39577425],
         ...,
         [3.21722984, 1.31997609],
         [3.23926163, 1.31692803],
         [3.22620964, 1.32057881]],
 
        ...,
 
        [[3.89128351, 2.25793195],
         [3.79784536, 2.07032108],
         [3.76586294, 1.93990648],
         ...,
         [3.76935387, 1.57853842],
         [3.73729277, 1.56849384],
         [3.7743268 , 1.56140959]],
 
        [[6.20525122, 

In [ ]:
batch_data, result

((<tf.Tensor: shape=(12, 126, 8), dtype=float32, numpy=
  array([[[ 4.22154129e-01,  5.78526735e-01,  6.94468319e-01, ...,
            7.02226639e-01,  1.66043854e+00,  1.39849818e+00],
          [ 2.72120357e-01,  4.73401397e-01,  6.11810982e-01, ...,
            8.59434485e-01,  1.87758172e+00,  1.46209788e+00],
          [ 1.50553852e-01,  3.85598093e-01,  5.29221356e-01, ...,
            6.16452217e-01,  2.23761201e+00,  1.47328079e+00],
          ...,
          [-1.35168362e+00, -9.07790482e-01, -3.40614229e-01, ...,
           -1.41744688e-01, -1.11088246e-01,  5.15854120e-01],
          [-1.36484730e+00, -9.87687826e-01, -4.50251132e-01, ...,
           -1.22196287e-01, -1.08352743e-01,  5.82782269e-01],
          [-1.32851899e+00, -1.02780652e+00, -5.31191885e-01, ...,
           -1.40540302e-01, -6.97354749e-02,  6.40299082e-01]],
  
         [[-7.60377884e-01, -1.01712239e+00, -1.06724250e+00, ...,
           -2.09264055e-01, -8.05742204e-01,  2.37559617e-01],
          [-7.1

In [ ]:
batch_data[-4][-2]

<tf.Tensor: shape=(126, 13), dtype=float32, numpy=
array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)>